In [1]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
import requests
import json
from datetime import *

StatementMeta(, 96ba2647-4a72-4a33-bca3-aef2e7bf521e, 3, Finished, Available, Finished)

In [2]:
# Leitura dos arquivos Parquet na pasta Novos
df = spark.read.parquet(f"abfss://Medallion_BCB@onelake.dfs.fabric.microsoft.com/Lakehouse_Bronze.Lakehouse/Files/Cotacoes/Novos/*.parquet")

df.createOrReplaceTempView("df")

df = spark.sql("""
    SELECT 
        cotacaoCompra AS Cotacao,
        CAST(dataHoraCotacao AS DATE) AS Data,
        moeda AS Moeda
    FROM
        df
    ORDER BY Data ASC
""").dropDuplicates(["Moeda", "Data"])

display(df)


StatementMeta(, 96ba2647-4a72-4a33-bca3-aef2e7bf521e, 4, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 5d45e015-172c-46f7-92fc-4c97e946d1e9)

In [3]:
%%sql

-- Cria uma tabela no lake se não existir
CREATE TABLE IF NOT EXISTS Lakehouse_Silver.cotacoes (
    Cotacao DOUBLE,
    Data DATE,
    Moeda STRING
)
USING DELTA
PARTITIONED BY (Moeda)

StatementMeta(, 96ba2647-4a72-4a33-bca3-aef2e7bf521e, 5, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [4]:
# Executa o merge na tabela cotacoes

df.createOrReplaceTempView("df_novos")

spark.sql("""
    MERGE INTO Lakehouse_Silver.cotacoes AS e
    USING (
        SELECT
            Cotacao,
            Data,
            Moeda
        FROM
            df_novos
    ) as n
    ON e.Moeda = n.Moeda
        AND e.Data = n.Data
    WHEN NOT MATCHED THEN
        INSERT (Cotacao, Data, Moeda)
        VALUES (n.Cotacao, n.Data, n. Moeda)
""")

StatementMeta(, 96ba2647-4a72-4a33-bca3-aef2e7bf521e, 6, Finished, Available, Finished)

DataFrame[num_affected_rows: bigint, num_updated_rows: bigint, num_deleted_rows: bigint, num_inserted_rows: bigint]

In [5]:
%%sql
-- Contagem de linhas
SELECT COUNT(*) FROM Lakehouse_Silver.cotacoes

StatementMeta(, 96ba2647-4a72-4a33-bca3-aef2e7bf521e, 7, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 1 fields>

In [6]:
# Movimentação dos arquivos parquet da pasta Novos para Carregados

from notebookutils import mssparkutils

origem = f"abfss://Medallion_BCB@onelake.dfs.fabric.microsoft.com/Lakehouse_Bronze.Lakehouse/Files/Cotacoes/Novos/"  
destino = f"abfss://Medallion_BCB@onelake.dfs.fabric.microsoft.com/Lakehouse_Bronze.Lakehouse/Files/Cotacoes/Carregados/" 

if not mssparkutils.fs.exists(destino):
    mssparkutils.fs.mkdirs(destino)

arquivos = mssparkutils.fs.ls(origem)

for arquivo in arquivos:
    caminho_origem = arquivo.path
    nome_arquivo = arquivo.name
    caminho_destino = f"{destino}{nome_arquivo}"
    
    mssparkutils.fs.mv(caminho_origem, caminho_destino)

StatementMeta(, 96ba2647-4a72-4a33-bca3-aef2e7bf521e, 8, Finished, Available, Finished)